In [ ]:
!pip install transformers datasets evaluate

In [ ]:
import collections
import pickle
from os import path

import numpy as np
from tqdm.auto import tqdm

from datasets import load_dataset

import evaluate

from transformers import AutoTokenizer
from transformers import create_optimizer
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering

## Loading Dataset

In [ ]:
squad_train = load_dataset("squad", split="train[:1000]")
squad_train = squad_train.train_test_split(test_size=0.1)

squad_test = load_dataset("squad", split="validation")

## Loading Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

## Preprocessing Dataset

In [ ]:
def preprocess_train_set(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
def preprocess_test_set(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
train_tokenized = squad_train['train'].map(preprocess_train_set, batched=True, remove_columns=squad_train["train"].column_names)
validation_tokenized = squad_train['test'].map(preprocess_train_set, batched=True, remove_columns=squad_train['test'].column_names)

test_tokenized = squad_test.map(preprocess_test_set, batched=True, remove_columns=squad_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

## Model Building

### Hyperparameters

In [ ]:
batch_size = 16
num_epochs = 3
total_train_steps = (len(train_tokenized) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.compile(optimizer=optimizer)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_59', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No lo

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained('/content/drive/MyDrive/QuestionAnsweringModels/QA_distilbert_base_uncased_v1')

All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at /content/drive/MyDrive/QuestionAnsweringModels/QA_distilbert_base_uncased_v1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    train_tokenized,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_dataset = model.prepare_tf_dataset(
    validation_tokenized,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16
)

tf_test_dataset = model.prepare_tf_dataset(
    test_tokenized,
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

## Training

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
56/56 [==============================] - 43s 678ms/step - loss: 4.9567 - val_loss: 4.0327
Epoch 2/3
56/56 [==============================] - 37s 668ms/step - loss: 3.5323 - val_loss: 3.5415
Epoch 3/3
56/56 [==============================] - 38s 681ms/step - loss: 2.8849 - val_loss: 3.4148


## Making Predictions

In [ ]:
predictions = model.predict(tf_test_dataset)

673/673 [==============================] - 153s 222ms/step


## Saving Models

In [ ]:
model.save_pretrained('distilbert_base_uncased_finetunedSquad_savepretrained_v0')

## Evaluating

In [ ]:
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples, n_best=20, max_answer_length=30):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers), predicted_answers

In [ ]:
scores, predicted_answers = compute_metrics(
    predictions["start_logits"],
    predictions["end_logits"],
    test_tokenized,
    squad_test,
)

  0%|          | 0/10570 [00:00<?, ?it/s]

In [ ]:
scores

{'exact_match': 76.5279091769158, 'f1': 84.77109624156209}

## Saving Predicted Answers

In [ ]:
with open(path.join('..', 'Outputs', 'predicted_answers.pkl')) as f:
    pickle.dump(predicted_answers, f)